In [2]:
# Group Number: 41
# Roll number: 21CS30064 Name: Ishaan Sinha  
# Roll number: 23BM6JP27 Name: Manas Bhat    
# Project Code: MCNB
# Project Title: Mushroom Classification using Naive Bayes Algorithm

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

C:\Users\ishaa\AppData\Local\Temp\ipykernel_9908\313998530.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
class NaiveBayesMushroomClassifier:
    def __init__(self, bins=20):
        self.bins = bins
        self.bin_edges = []
        self.prior = None
        self.labels = None
        self.counts = None
        self.likelihood_prob = None
        self.hist_class = None

    def binning(self, X):
        self.bin_edges = []
        for i in range(X.shape[1]):
            self.bin_edges.append(np.histogram_bin_edges(X[:, i], bins=self.bins))

    def prior_prob(self, y):
        labels, counts = np.unique(y, return_counts=True)
        self.prior = counts / len(y)
        self.labels = labels
        self.counts = counts

    def split_class(self, X, y):
        x_train_class = []

        for i in self.labels:
            x_train_class.append(X[y == i])

        return x_train_class

    def bins_hist(self, X):
        bins_hist = []

        for i in range(X.shape[1]):
            bins_hist.append([np.histogram(feature, bins=self.bin_edges[i])[0] for feature in X.T])

        return bins_hist

    def likelihood(self, x_hist_class):
        likelihood_prob = []

        for i in range(len(self.labels)):
            l = []
            for j in range(len(x_hist_class[i])):
                temp_l = []
                for k in range(len(x_hist_class[i][j])):
                    temp_l.append((x_hist_class[i][j][k] + 1) / (self.counts[i] + self.bins))
                l.append(temp_l)
            likelihood_prob.append(l)
        return likelihood_prob

    def fit(self, X, y):
        self.binning(X)
        self.prior_prob(y)
        x_train_class = self.split_class(X, y)
        x_hist_class = [self.bins_hist(x) for x in x_train_class]
        self.hist_class = x_hist_class
        self.likelihood_prob = self.likelihood(x_hist_class)

    def posterior(self, x):
        posterior_prob = []

        for i in range(len(self.labels)):
            l = []
            for j in range(len(x)):
                for k in range(len(self.bin_edges[j]) - 1):
                    if self.bin_edges[j][k] <= x[j] < self.bin_edges[j][k + 1]:
                        l.append(self.likelihood_prob[i][j][k])
                        break

            posterior_prob.append(np.prod(l) * self.prior[i])

        return self.labels[np.argmax(posterior_prob)]

    def predict(self, X_test):
        preds = [self.posterior(x) for x in X_test]
        return preds

In [20]:
# Load the mushroom dataset
data = pd.read_csv('mushrooms.csv')
#shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

In [21]:
# Preprocess the data
data_encoded = pd.get_dummies(data)
X = data_encoded.drop(columns=['class_p', 'class_e']).values
y = data_encoded['class_p'].values

In [22]:
# Split the dataset into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Instantiate and train the Naive Bayes classifier
nb_classifier = NaiveBayesMushroomClassifier()
nb_classifier.fit(X_train, y_train)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_9908\3764672876.py:14: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  self.bin_edges.append(np.histogram_bin_edges(X[:, i], bins=self.bins))
C:\Users\ishaa\AppData\Local\Temp\ipykernel_9908\3764672876.py:34: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  bins_hist.append([np.histogram(feature, bins=self.bin_edges[i])[0] for feature in X.T])


In [24]:
# Make predictions on the test set
y_pred_custom = nb_classifier.predict(X_test)

In [25]:
# Evaluate the custom Naive Bayes classifier
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print("Custom Naive Bayes Classifier Accuracy:", accuracy_custom)

Custom Naive Bayes Classifier Accuracy: 0.5175384615384615


In [26]:
# Generate classification report
report = classification_report(y_test, y_pred_custom, target_names=['edible', 'poisonous'])

c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [12]:
# Print classification report
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

      edible       0.52      1.00      0.68       843
   poisonous       0.00      0.00      0.00       782

    accuracy                           0.52      1625
   macro avg       0.26      0.50      0.34      1625
weighted avg       0.27      0.52      0.35      1625



In [15]:
# Instantiate and train the Actual Naive Bayes
import sklearn.naive_bayes
nb_classifier = sklearn.naive_bayes.CategoricalNB()
nb_classifier.fit(X_train, y_train)

CategoricalNB()

In [16]:
# Make predictions on the test set
y_pred_custom = nb_classifier.predict(X_test)

In [17]:
# Evaluate the custom Naive Bayes classifier
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print("Custom Naive Bayes Classifier Accuracy:", accuracy_custom)

Custom Naive Bayes Classifier Accuracy: 0.936


In [18]:
# Generate classification report
report = classification_report(y_test, y_pred_custom, target_names=['edible', 'poisonous'])

In [19]:
# Print classification report
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

      edible       0.90      0.98      0.94       843
   poisonous       0.98      0.89      0.93       782

    accuracy                           0.94      1625
   macro avg       0.94      0.93      0.94      1625
weighted avg       0.94      0.94      0.94      1625

